In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import pandas as pd
import tensorflow as tf
from operator import itemgetter
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('taal/Trained_Audio_Labels_taal.csv')

In [3]:
df

,filename,onset,offset,class
0,mixed_audio0a.wav,0.00000,4.08306,silence
1,mixed_audio0a.wav,4.08306,5.08306,music
2,mixed_audio0a.wav,5.08306,6.90256,silence
3,mixed_audio0a.wav,6.90256,9.90256,speech
4,mixed_audio0a.wav,9.90256,10.00000,silence
...,...,...,...,...
1187,mixed_audio9a.wav,0.00000,3.35275,silence
1188,mixed_audio9a.wav,3.35275,6.35275,speech
1189,mixed_audio9a.wav,6.35275,6.71350,silence
1190,mixed_audio9a.wav,6.71350,9.71350,music


In [4]:
df.rename({'class':'class_name'},axis=1,inplace=True)

### Spliting the data into Training and Testing/Validation data

In [5]:
df_train=df.sample(frac=0.85,random_state=200)

In [6]:
df_test = df.drop(df_train.index)

In [7]:
df_train

,filename,onset,offset,class_name
716,mixed_audio22a.wav,5.42231,9.42231,music
599,mixed_audio209a.wav,0.26362,4.26362,speech
402,mixed_audio173a.wav,9.47569,10.00000,silence
1019,mixed_audio69a.wav,1.00306,4.00306,music
191,mixed_audio134a.wav,4.29487,4.62350,silence
...,...,...,...,...
563,mixed_audio202a.wav,0.00000,2.72694,silence
847,mixed_audio37a.wav,5.73744,6.23075,silence
197,mixed_audio135a.wav,7.45525,9.45525,music
325,mixed_audio15a.wav,0.24063,3.24063,music


In [8]:
df_test

,filename,onset,offset,class_name
0,mixed_audio0a.wav,0.00000,4.08306,silence
10,mixed_audio101a.wav,0.00000,0.74662,silence
15,mixed_audio102a.wav,0.00000,0.55350,silence
17,mixed_audio102a.wav,2.55350,5.83256,silence
29,mixed_audio104a.wav,9.96075,10.00000,silence
...,...,...,...,...
1162,mixed_audio95a.wav,0.00000,1.50537,silence
1163,mixed_audio95a.wav,1.50537,4.50537,music
1167,mixed_audio96a.wav,0.00000,1.89263,silence
1175,mixed_audio97a.wav,8.06525,9.06525,music


In [9]:
x_train = []
y_train = []

In [10]:
filename_train = df_train.filename.values
classname_train = df_train.class_name.values
onset_train = df_train.onset.values
offset_train = df_train.offset.values

In [11]:
filename_train

array(['mixed_audio22a.wav', 'mixed_audio209a.wav', 'mixed_audio173a.wav',
       ..., 'mixed_audio135a.wav', 'mixed_audio15a.wav',
       'mixed_audio82a.wav'], dtype=object)

In [12]:
onset_train

array([5.42231, 0.26362, 9.47569, ..., 7.45525, 0.24063, 2.12462])

In [13]:
len(classname_train)

1013

### MFCC Features and Label Encoding

In [14]:
z = np.array([])
Fs = 16000
curr_on_train = 0
curr_off_train = 0
for i in range(len(filename_train)):
    #print(i)
    max_range_on = int(round(onset_train[i]*Fs,0))
    max_range_off = int(round(offset_train[i]*Fs,0))
    curr_on_train = max_range_on
    curr_off_train = max_range_off
    cnt =0
    z,tsr = librosa.load('taal/'+filename_train[i],sr=16000)
    temp2 = z[curr_on_train:curr_off_train]
    mfcc = librosa.feature.mfcc(y=temp2,sr=16000,n_mfcc=20)
    mfcc_n = []
    for j in range (mfcc.shape[0]):
        mfcc_n.append(mfcc[j].mean())
    mfcc_n = np.array(mfcc_n)
    x_train.append(mfcc_n)
    
    if classname_train[i]=='silence':
        y_train.append([0,0,1])
    if classname_train[i]=='speech':
        y_train.append([1,0,0])
    if classname_train[i]=='music':
        y_train.append([0,1,0])    
    

In [15]:
len(x_train)

1013

In [16]:
filename_test = df_test.filename.values
classname_test = df_test.class_name.values
onset_test = df_test.onset.values
offset_test = df_test.offset.values

In [17]:
x_test = []
y_test = []

In [18]:
x = np.array([])
Fs = 16000
curr_on_test = 0
curr_off_test = 0
sumi=0

for i in range(0,len(filename_test)):
    #print(i)
    x,ysr = librosa.load('taal/'+filename_test[i],sr=Fs)
    max_range_on = int(round(onset_test[i]*Fs,0))
    max_range_off = int(round(offset_test[i]*Fs,0))
    curr_on_test = max_range_on
    curr_off_test = max_range_off
    j=curr_on_test  
    cnt =0
    while j<curr_off_test:
        cnt +=1
        end = min(j+800,curr_off_test)
#         print(" j and end "+ str(j)+ " "+ str(end))
        temp2 = x[j:end]
        mfcc = librosa.feature.mfcc(y=temp2,sr=16000,n_mfcc=20)
        mfcc_n =[]
        for k in range(mfcc.shape[0]):
            mfcc_n.append(mfcc[k].mean())
        mfcc_n = np.array(mfcc_n)
        x_test.append(mfcc_n)
        if classname_test[i]=='silence':
            y_test.append([0,0,1])
        elif classname_test[i]=='speech':
            y_test.append([1,0,0])
        elif classname_test[i]=='music':
            y_test.append([0,1,0])
        j =end
    
    sumi +=cnt    
    if i==0:
        print('on and off ' + str(curr_on_test) + " " + str(curr_off_test) + " "+str(sumi))
print("sUMI " + str(sumi))
 

on and off 0 65329 82
sUMI 7605


In [19]:
X_train = np.array(x_train)

In [20]:
X_train.shape

(1013, 20)

In [21]:
X_test = np.array(x_test)

In [22]:
X_test.shape

(7605, 20)

In [23]:
Y_train = np.array(y_train)

In [24]:
Y_train.shape

(1013, 3)

In [25]:
Y_test = np.array(y_test)

In [26]:
Y_test.shape

(7605, 3)

In [27]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, BatchNormalization, Flatten, LSTM

### Building a Simple Neural Network Model

In [53]:
model = Sequential()
model.add(Dense(64,activation='relu',input_shape =(20,)))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dense(3,activation='softmax'))

In [57]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 64)                1344      
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)               

In [58]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [60]:
model.fit(X_train, Y_train, epochs=100, batch_size=16, validation_data=(X_test, Y_test), shuffle=False)

Epoch 1/100
64/64 [==============================] - 1s 8ms/step - loss: 7.9977 - accuracy: 0.6041 - val_loss: 1.1414 - val_accuracy: 0.7858
Epoch 2/100
64/64 [==============================] - 0s 6ms/step - loss: 1.8750 - accuracy: 0.7256 - val_loss: 0.5468 - val_accuracy: 0.8592
Epoch 3/100
64/64 [==============================] - 0s 5ms/step - loss: 1.1118 - accuracy: 0.7749 - val_loss: 0.4010 - val_accuracy: 0.8698
Epoch 4/100
64/64 [==============================] - 0s 6ms/step - loss: 0.8360 - accuracy: 0.7887 - val_loss: 0.5092 - val_accuracy: 0.8565
Epoch 5/100
64/64 [==============================] - 0s 6ms/step - loss: 0.6310 - accuracy: 0.8282 - val_loss: 0.3893 - val_accuracy: 0.8771
Epoch 6/100
64/64 [==============================] - 0s 6ms/step - loss: 0.3496 - accuracy: 0.8855 - val_loss: 0.3349 - val_accuracy: 0.8969
Epoch 7/100
64/64 [==============================] - 0s 6ms/step - loss: 0.3392 - accuracy: 0.8963 - val_loss: 0.4187 - val_accuracy: 0.8944
Epoch 8/100
6

Epoch 59/100
64/64 [==============================] - 0s 5ms/step - loss: 0.0341 - accuracy: 0.9931 - val_loss: 0.8603 - val_accuracy: 0.9381
Epoch 60/100
64/64 [==============================] - 0s 5ms/step - loss: 0.0163 - accuracy: 0.9941 - val_loss: 1.0726 - val_accuracy: 0.9350
Epoch 61/100
64/64 [==============================] - 0s 6ms/step - loss: 0.0145 - accuracy: 0.9961 - val_loss: 1.2078 - val_accuracy: 0.9314
Epoch 62/100
64/64 [==============================] - 0s 5ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 1.0604 - val_accuracy: 0.9377
Epoch 63/100
64/64 [==============================] - 0s 5ms/step - loss: 0.0126 - accuracy: 0.9961 - val_loss: 1.1671 - val_accuracy: 0.9344
Epoch 64/100
64/64 [==============================] - 0s 5ms/step - loss: 0.0100 - accuracy: 0.9970 - val_loss: 1.8662 - val_accuracy: 0.9191
Epoch 65/100
64/64 [==============================] - 0s 5ms/step - loss: 0.0114 - accuracy: 0.9970 - val_loss: 1.4741 - val_accuracy: 0.9294
Epoch 

In [62]:
model.save('tfModel')

INFO:tensorflow:Assets written to: tfModel\assets


In [63]:
loss, acc = model.evaluate(X_test, Y_test, verbose=0)

In [64]:
loss

1.3445290327072144

In [65]:
acc

0.9338592886924744